# Lab 4

## Design and Create Database (SQLite3)

In this Lab section, I will build a database of several tables that contain different infomation of several enzymes from different organisms and pathways.

According to the Lab Assgnments aquirement, I have designed and created 3 tables in a databases called **genes**, **enzymes** and **pathways**.

In [1]:
import sqlite3

# connect database
conn = sqlite3.connect('lab4.db')
c = conn.cursor()

# create tables: gene, pathway, enzyme
with conn:
    c.execute("""CREATE TABLE IF NOT EXISTS genes (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                    name TEXT,
                                    description TEXT,
                                    organism TEXT,
                                    nt_seq TEXT,
                                    chromosome TEXT,
                                    start INTEGER,
                                    end INTEGER,
                                    tran_seq TEXT);""")
    c.execute("""CREATE TABLE IF NOT EXISTS pathways (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                        name TEXT,
                                        description TEXT);""")
    c.execute("""CREATE TABLE IF NOT EXISTS enzymes (id INTEGER PRIMARY KEY AUTOINCREMENT,
                                        name TEXT,
                                        function TEXT,
                                        ec_num INT);""")

## Searching databases for data, and inserting into my own

The organism I choose are *Saccharomyces cerevisiae*, *Homo Sapiens* and *Escherichia coli*. The first two are eukaryotes, and the last one is a prokaryote.

The pathways included in the database are **glycolysis**, **TCA cycle** and **pentose phosphate pathway**. Dscriptions and enzymes involve in each pathway are aquired from KEGG database:

1. [glycolysis](https://www.genome.jp/kegg-bin/show_pathway?map00010), enzymes: 2.7.1.1(hexokinase) 5.3.1.9(glucose-6-phosphate isomerase), 5.3.1.1(triosephosphate isomerase) and 4.1.2.13(Fructose-bisphosphate aldolase).

2. [TCA cycle](https://www.genome.jp/kegg-bin/show_pathway?map00020), enzymes: 2.3.3.1(Citrate synthase), 4.2.1.3(Aconitase), 1.1.1.42(Isocitrate dehydrogenase), 1.2.4.2(α-Ketoglutarate dehydrogenase)

3. [pentose phophate pathway](https://www.genome.jp/kegg-bin/show_pathway?map00030), enzymes: 1.1.1.49(glucose 6-phosphate dehydrogenase), 3.1.1.31(6-phosphogluconolactonase), 1.1.1.44(6-phosphogluconate dehydrogenase), 5.3.1.6(Ribose-5-phosphate isomerase)

Automated searching of these enzyme are performed by `Bio.Entrez`. This package implemented the NCBI searching API, and can be used to perform inqury similar to the search done on the NCBI webpage.

The initial attempt to get all the information for an enzyme was through the `nucleotide` database. However, this database has significant amount of irrelevant nucleotide segments. The return of the inquiry sometimes can be a whole genome, partial or mRNA segments, or completely irrelevant segments.

In order to solve this problem, we have turned to the `gene` database, where data are more unique. However, the main disvantage of this database is that the result is not in `fasta` or `genbank` format. We need to parse the return text manually.

In order to do this, I have summarized the codes for parsing the return text of `gene` and then how to use these parsed information to find desired gene using `nuccore` database. I have named this module [**ABWSE**, this link lead to a jupyter notebook describes this module](https://github.com/RainLiuX/be131/blob/master/lab4/abwseABWSE%20module.ipynb) (A Better Way to Search for an Enzyme). This would significantly increase the accuracy of *gene hunting* and keep the codes in a neat and concise manner.

This API takes the name or the EC number of the enzyme, and the organism as parameter, and return a series of information, as leasted below:

In [2]:
pathways = ['glycolysis', 'TCA cycle', 'pentose phosphata']
species = ['Saccharomyces cerevisiae', 'Homo Sapiens', 'Escherichia coli']
enzymes = ['Hexokinase', 'Glucose-6-phosphate isomerase', 'Triosephosphate isomerase', 'Fructose-bisphosphate aldolase',\
          'Citrate synthase', 'Aconitase', 'Isocitrate dehydrogenase', 'α-Ketoglutarate dehydrogenase',\
          'glucose 6-phosphate dehydrogenase', '6-phosphogluconolactonase', '6-phosphogluconate dehydrogenase', 'Ribose-5-phosphate isomerase']

# create the pathway database
pathway_descriptions = ["Glycolysis is the metabolic pathway that converts glucose, into pyruvate.",\
                       "TCA cycle is a series of chemical reactions used by all aerobic organisms to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins into adenosine triphosphate (ATP) and carbon dioxide",\
                       "Pentose phosphate pathway is a metabolic pathway parallel to glycolysis. It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate."]
for (index, pathway) in enumerate(pathways):
    with conn:
        query_str = """
                    INSERT INTO pathways (name, description)
                    VALUES ('{}', '{}')
                    """.format(pathway, pathway_descriptions[index])
        c.execute(query_str)

Notice that even for the same enzyme, the name or the EC number is different in different species, especially in prokaryotic species such as *Escherichia coli*. This could due to the incomplete of prokaryotic gene database, or due to the slight discrepency between the pathways found in eukaryotes and prokaryotes.

In [3]:
from abwse import abwse
import re

enzyme_info_list = []
for enz in enzymes :
    for spe in species :
        # repeat if encounter internet error
        while True:
            try:
                enzyme_info = abwse.ezsearch(enz, spe, 'rainl199922@berkeley.edu', rettype='dict')
            except OSError as e:
                print(e)
            else:
                break
        # adding returned enzyme info into a list
        enzyme_info_list.append(enzyme_info)
        # inserting returned enzyme info into database, gene table
        with conn:
            query_str = """
                        INSERT INTO genes (name, description, organism, nt_seq, chromosome, start, end, tran_seq)
                        VALUES ('{}', '{}', '{}', '{}', '{}', {}, {}, '{}');
                        """.format(enzyme_info['name'],\
                                   re.sub(r"'", r"''", enzyme_info['description']),\
                                   enzyme_info['species'],\
                                   enzyme_info['nt_seq'],\
                                   enzyme_info['chromosome'],\
                                   enzyme_info['start'],\
                                   enzyme_info['end'],\
                                   None if enzyme_info['tran_seq'] is None else enzyme_info['tran_seq'][0])
            c.execute(query_str)
        # if the first species of the enzyme, insert into enzyme table
        if spe == species[0]:
            with conn:
                query_str = """
                            INSERT INTO enzymes (name, function, ec_num)
                            VALUES ('{}', '{}', '{}');
                            """.format(enzyme_info['name'],\
                                      re.sub(r"'", r"''", enzyme_info['description']),\
                                      enzyme_info['EC'])
                c.execute(query_str)
print(abwse.eztable(*enzyme_info_list, width=100))

+----------------+-------------------------+-------------------+----------------+------------------+
|      name      |       enzyme_name       |      species      |     start      |       end        |
+----------------+-------------------------+-------------------+----------------+------------------+
|      HXK2      |      hexokinase 2       | Saccharomyces cer |     23935      |      25395       |
|                |                         |   evisiae S288C   |                |                  |
+----------------+-------------------------+-------------------+----------------+------------------+
|  description   |       chromosome        |        EC         |     nt_seq     |     tran_seq     |
+----------------+-------------------------+-------------------+----------------+------------------+
| Hexokinase iso |           VII           |      2.7.1.1      | ATGGTTCATTTAGG | 1 types; MVHLGPK |
| enzyme 2; phos |                         |                   | TCCAAAAAAACCAC | KPQARKGSM

Printing out all data in 3 databases:

In [4]:
c.execute('SELECT * FROM genes;')
print(c.fetchall())

[(1, 'HXK2', 'Hexokinase isoenzyme 2; phosphorylates glucose in cytosol; predominant hexokinase during growth on glucose; represses expression of HXK1, GLK1, induces expression of its own gene; antiapoptotic; phosphorylation/dephosphorylation at Ser14 by kinase Snf1p, phosphatase Glc7p-Reg1p regulates nucleocytoplasmic shuttling of Hxk2p; functions downstream of Sit4p in control of cell cycle, mitochondrial function, oxidative stress resistance, chronological lifespan; has paralog HXK1', 'Saccharomyces cerevisiae S288C', 'ATGGTTCATTTAGGTCCAAAAAAACCACAAGCCAGAAAGGGTTCCATGGCCGATGTGCCAAAGGAATTGATGCAACAAATTGAGAATTTTGAAAAAATTTTCACTGTTCCAACTGAAACTTTACAAGCCGTTACCAAGCACTTCATTTCCGAATTGGAAAAGGGTTTGTCCAAGAAGGGTGGTAACATTCCAATGATTCCAGGTTGGGTTATGGATTTCCCAACTGGTAAGGAATCCGGTGATTTCTTGGCCATTGATTTGGGTGGTACCAACTTGAGAGTTGTCTTAGTCAAGTTGGGCGGTGACCGTACCTTTGACACCACTCAATCTAAGTACAGATTACCAGATGCTATGAGAACTACTCAAAATCCAGACGAATTGTGGGAATTTATTGCCGACTCTTTGAAAGCTTTTATTGATGAGCAATTCCCACAAGGTATCTCTGAGCCAATTCCATTGGGTTTCACCTTTT

In [5]:
c.execute('SELECT * FROM enzymes;')
print(c.fetchall())

[(1, 'HXK2', 'Hexokinase isoenzyme 2; phosphorylates glucose in cytosol; predominant hexokinase during growth on glucose; represses expression of HXK1, GLK1, induces expression of its own gene; antiapoptotic; phosphorylation/dephosphorylation at Ser14 by kinase Snf1p, phosphatase Glc7p-Reg1p regulates nucleocytoplasmic shuttling of Hxk2p; functions downstream of Sit4p in control of cell cycle, mitochondrial function, oxidative stress resistance, chronological lifespan; has paralog HXK1', '2.7.1.1'), (2, 'PGI1', 'Glycolytic enzyme phosphoglucose isomerase; catalyzes the interconversion of glucose-6-phosphate and fructose-6-phosphate; required for cell cycle progression and completion of the gluconeogenic events of sporulation', '5.3.1.9'), (3, 'TPI1', "Triose phosphate isomerase, abundant glycolytic enzyme; mRNA half-life is regulated by iron availability; transcription is controlled by activators Reb1p, Gcr1p, and Rap1p through binding sites in the 5' non-coding region; inhibition of T

In [6]:
c.execute('SELECT * FROM pathways;')
print(c.fetchall())

[(1, 'glycolysis', 'Glycolysis is the metabolic pathway that converts glucose, into pyruvate.'), (2, 'TCA cycle', 'TCA cycle is a series of chemical reactions used by all aerobic organisms to release stored energy through the oxidation of acetyl-CoA derived from carbohydrates, fats, and proteins into adenosine triphosphate (ATP) and carbon dioxide'), (3, 'pentose phosphata', 'Pentose phosphate pathway is a metabolic pathway parallel to glycolysis. It generates NADPH and pentoses (5-carbon sugars) as well as ribose 5-phosphate.')]


Drop all tables in case we need to start everything all over again.

In [7]:
# import sqlite3

# # connect database
# conn = sqlite3.connect('lab4.db')
# c = conn.cursor()
# with conn:
#     c.execute("DROP TABLE IF EXISTS genes;")
#     c.execute("DROP TABLE IF EXISTS enzymes;")
#     c.execute("DROP TABLE IF EXISTS pathways;")